1\. This query is used to find the first and last school years that codes are used in incidents. It can be adapted for various tables.A

In [2]:
USE DIRS;
WITH
    LastDateCTE
    AS
    (
        SELECT
            t.altEdServicesTypeID value_column,
            CAST(i.incidentDate AS DATE) AS incident_date,
            ROW_NUMBER() OVER (PARTITION BY t.altEdServicesTypeID ORDER BY i.incidentDate DESC) AS rnDesc,
            ROW_NUMBER() OVER (PARTITION BY t.altEdServicesTypeID ORDER BY i.incidentDate ASC) AS rnAsc
        from dbo.altEdServicesType t
            JOIN dbo.DisciplinaryAction o ON t.altEdServicesTypeID = o.altEdServicesTypeID
            JOIN dbo.Incident i ON o.incidentid = i.incidentid
    ),
    SchoolYearCTE
    AS
    (
        SELECT
            value_column,
            incident_date,
            RIGHT(CAST(CASE WHEN MONTH(incident_date) >= 7 THEN YEAR(incident_date) ELSE YEAR(incident_date) - 1 END AS VARCHAR(4)), 2) + '-' +
        RIGHT(CAST(CASE WHEN MONTH(incident_date) >= 7 THEN YEAR(incident_date) + 1 ELSE YEAR(incident_date) END AS VARCHAR(4)), 2) AS school_year,
            rnDesc,
            rnAsc
        FROM LastDateCTE
    )
SELECT
    a.value_column,
    b.school_year AS first_school_year,
    CASE WHEN a.school_year = '23-24' THEN '' ELSE a.school_year END AS last_school_year,
    b.incident_date AS first_incident_date,
    a.incident_date AS last_incident_date
FROM SchoolYearCTE a
    JOIN SchoolYearCTE b ON (a.value_column = b.value_column) AND (a.rnDesc = 1) AND (b.rnAsc = 1)
ORDER BY a.value_column;

(15 rows affected)

Total execution time: 00:00:00.473

value_column,first_school_year,last_school_year,first_incident_date,last_incident_date
1,08-09,,2008-09-04,2024-05-14
2,08-09,,2008-09-05,2024-05-21
3,08-09,,2008-09-02,2024-06-04
4,08-09,,2008-10-01,2024-06-04
5,08-09,,2008-09-02,2024-06-03
6,08-09,,2008-09-08,2024-05-30
7,08-09,22-23,2008-10-08,2023-04-12
8,08-09,,2008-07-09,2024-05-20
9,08-09,,2008-09-10,2024-05-23
10,08-09,,2008-10-13,2024-04-24


2\. This query selects the records from the Incident offender table where the combination of incidentID and offenderID appear more than once.

As you can see, when this is the case, the incidentTypeID is causing the duplication.

In [5]:
SELECT [incidentOffenderID]
      ,[incidentID]
      ,[offenderID]
      ,[incidentTypeID]
      ,[offenseTypeID]
      ,[lastUserModified]
      ,[lastDateModified]
FROM [DIRS].[dbo].[IncidentOffender] main
WHERE EXISTS (
    SELECT 1 --We are only interested in the existence of rows in the subquery, not the actual data, so we can use SELECT 1
    FROM [DIRS].[dbo].[IncidentOffender] AS sub
    WHERE sub.incidentID = main.incidentID
    AND sub.offenderID = main.offenderID
    GROUP BY sub.incidentID, sub.offenderID
    HAVING COUNT(*) > 1
)
ORDER BY lastDateModified DESC;

(31873 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:00.664

incidentOffenderID,incidentID,offenderID,incidentTypeID,offenseTypeID,lastUserModified,lastDateModified
624890,255946,98942,12,3,CMJOHN08,2012-08-21 16:34:23.000
624889,255946,98942,3,1,CMJOHN08,2012-08-21 16:34:23.000
624866,255941,13558,21,NULL,CMJOHN08,2012-08-21 14:22:43.000
624865,255941,13558,12,4,CMJOHN08,2012-08-21 14:22:43.000
624834,255926,9686,21,NULL,CMJOHN08,2012-08-21 11:02:10.000
624833,255926,9686,8,NULL,CMJOHN08,2012-08-21 11:02:10.000
624831,255925,44026,18,NULL,CMJOHN08,2012-08-21 09:48:14.000
624830,255925,44026,10,NULL,CMJOHN08,2012-08-21 09:48:14.000
624812,255916,73419,21,NULL,CMJOHN08,2012-08-21 09:07:21.000
624811,255916,73419,12,3,CMJOHN08,2012-08-21 09:07:21.000


3\. This query selects the records from the Incident offender table where the combination of incidentID and offenderID appear more than once.

In [3]:
SELECT TOP(100) *
FROM [RDMESSA].[RPT].[uvw_AcademicDetail] main
WHERE EXISTS (
    SELECT 1 --We are only interested in the existence of rows in the subquery, not the actual data, so we can use SELECT 1
    FROM [RDMESSA].[RPT].[uvw_AcademicDetail] AS sub
    WHERE sub.FiscalYear = main.FiscalYear
    AND sub.MARSSNumber = main.MARSSNumber
    AND sub.TestName = main.TestName
    AND sub.StateOrganizationID = main.StateOrganizationID
    GROUP BY sub.FiscalYear, sub.MARSSNumber, sub.TestName, sub.StateOrganizationID
    HAVING COUNT(*) > 1
)
ORDER BY FiscalYear DESC;
--[DIRS].[dbo].[Incident]

: Query failed: The operation was canceled.

Query was canceled by user

Total execution time: 01:25:37.041

4\. This query checks each column in a table for NULLs.

In [16]:
USE DIRS;

DECLARE @sql NVARCHAR(MAX);

CREATE TABLE #Results (
    ColumnName NVARCHAR(128),
    ContainsNulls CHAR(1)
);

DECLARE @columns CURSOR;
DECLARE @columnName NVARCHAR(128);

SET @columns = CURSOR FOR
SELECT c.name
FROM sys.columns c
JOIN sys.objects o ON c.object_id = o.object_id
WHERE o.name = 'IncidentDangerousWeapon' AND o.type = 'U'
ORDER BY c.column_id;

OPEN @columns;
FETCH NEXT FROM @columns INTO @columnName;

WHILE @@FETCH_STATUS = 0
BEGIN
    SET @sql = N'
    INSERT INTO #Results (ColumnName, ContainsNulls)
    SELECT ''' + @columnName + ''',
        CASE 
            WHEN EXISTS (
                SELECT 1 
                FROM INFORMATION_SCHEMA.COLUMNS
                WHERE TABLE_SCHEMA = ''dbo''
                  AND TABLE_NAME = ''IncidentDangerousWeapon''
                  AND COLUMN_NAME = ''' + @columnName + '''
                  AND IS_NULLABLE = ''YES''
            ) THEN 
                CASE 
                    WHEN EXISTS (
                        SELECT 1 
                        FROM dbo.IncidentDangerousWeapon 
                        WHERE COLUMNPROPERTY(OBJECT_ID(''dbo.IncidentDangerousWeapon''), ''' + @columnName + ''', ''AllowsNull'') = 1
                          AND ' + @columnName + ' IS NULL
                    ) THEN ''Y''
                    ELSE ''N''
                END
            ELSE ''N''
        END';

    EXEC sp_executesql @sql;

    FETCH NEXT FROM @columns INTO @columnName;
END

CLOSE @columns;
DEALLOCATE @columns;

SELECT * FROM #Results ORDER BY ColumnName;

DROP TABLE #Results;

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(10 rows affected)

Total execution time: 00:00:00.250

ColumnName,ContainsNulls
incidentDangerousWeaponID,N
incidentID,N
incidentOffenderID,Y
incidentTypeID,Y
incidentWeaponID,Y
lastDateModified,N
lastUserModified,N
wasUsedAsADangerousWeapon,Y
wasUsedAsADangerousWeaponFed,Y
wasUsedAsADangerousWeaponState,Y
